In [1]:
ls ./data

file_0.json    file_126.json  file_16.json   file_44.json   file_72.json
file_1.json    file_127.json  file_17.json   file_45.json   file_73.json
file_10.json   file_128.json  file_18.json   file_46.json   file_74.json
file_100.json  file_129.json  file_19.json   file_47.json   file_75.json
file_101.json  file_13.json   file_2.json    file_48.json   file_76.json
file_102.json  file_130.json  file_20.json   file_49.json   file_77.json
file_103.json  file_131.json  file_21.json   file_5.json    file_78.json
file_104.json  file_132.json  file_22.json   file_50.json   file_79.json
file_105.json  file_133.json  file_23.json   file_51.json   file_8.json
file_106.json  file_134.json  file_24.json   file_52.json   file_80.json
file_107.json  file_135.json  file_25.json   file_53.json   file_81.json
file_108.json  file_136.json  file_26.json   file_54.json   file_82.json
file_109.json  file_137.json  file_27.json   file_55.json   file_83.json
file_11.json   file_138.json  file_28.json   file_56

There are about 50 JSON files in the directory. Let's see what each JSON file contains.

In [2]:
cat data/file_0.json

{"name": "BR 7", "region_id": "region_107", "lat": 37.771557513543236, "station_id": "hub_516", "channel": "US-Bike-Sharing-Channel", "num_bikes_available": 3, "country_code": "US", "last_reported": 1495379589, "is_returning": 1, "rental_methods": ["KEY", "APPLEPAY", "ANDROIDPAY", "TRANSITCARD", "ACCOUNTNUMBER", "PHONE"], "is_installed": 1, "lon": -121.96133270859717, "num_bikes_disabled": 0, "is_renting": 1, "address": "USPS, San Ramon Valley Iron Horse Trail, San Ramon, California", "num_docks_available": 3, "publisher": "britebikes.socialbicycles.com"}
{"name": "San Ramon Test", "region_id": "region_122", "lat": 37.7714624826912, "station_id": "hub_662", "channel": "US-Bike-Sharing-Channel", "num_bikes_available": 2, "country_code": "US", "last_reported": 1495379589, "is_returning": 1, "rental_methods": ["KEY", "APPLEPAY", "ANDROIDPAY", "TRANSITCARD", "ACCOUNTNUMBER", "PHONE"], "is_installed": 1, "lon": -121.992872431289, "num_bikes_disabled": 0, "is_renting": 1, "address": "2401 Cr

In [3]:
from pyspark.sql.types import *

inputPath = "./data/"

# Since we know the data format already, let's define the schema to speed up processing (no need for Spark to infer schema)
jsonSchema = StructType([ StructField("last_reported", TimestampType(), True), 
                          StructField("name", StringType(), True),
                          StructField("station_id", StringType(), True),
                          StructField("region_id", StringType(), True),
                          StructField("publisher", StringType(), True),
                          StructField("lat", FloatType(), True),
                          StructField("lon", FloatType(), True),
                          StructField("country_code", StringType(), True),
                          StructField("num_bikes_available", IntegerType(), True),
                          StructField("num_docks_available", IntegerType(), True),
                          StructField("is_renting", IntegerType(), True),
                          StructField("is_returning", IntegerType(), True)
                        ])


## Static Streaming

In [4]:
# Static DataFrame representing data in the JSON files
staticInputDF = (
  spark
    .read
    .schema(jsonSchema)
    .json(inputPath)
)

#display(staticInputDF)
staticInputDF.count()

1533

Now we can compute the number of "open" and "close" actions with one hour windows. To do this, we will group by the `action` column and 1 hour windows over the `time` column.

In [5]:
from pyspark.sql.functions import *      # for window() function

staticCountsperCountryDF = (
  staticInputDF
    .groupBy(
       staticInputDF.country_code, 
       window(staticInputDF.last_reported, "1 minute"))    
    .count()
)
staticCountsperCountryDF.cache()

# Register the DataFrame as table 'static_counts'
staticCountsperCountryDF.createOrReplaceTempView("country_counts")

In [6]:
spark.sql("select country_code, sum(count) as total_count from country_counts group by country_code").show()

+------------+-----------+
|country_code|total_count|
+------------+-----------+
|          AU|         41|
|          CA|        119|
|          US|       1155|
+------------+-----------+



In [7]:
spark.sql("select * from country_counts where country_code = 'AU'").show()

+------------+--------------------+-----+
|country_code|              window|count|
+------------+--------------------+-----+
|          AU|[2017-05-21 17:13...|   41|
+------------+--------------------+-----+



How about a timeline of windowed counts?

In [8]:
spark.sql("select country_code, date_format(window.end, 'MMM-dd HH:mm') as time, count from country_counts order by time, country_code").show()

+------------+------------+-----+
|country_code|        time|count|
+------------+------------+-----+
|          US|May-21 17:13|    3|
|          AU|May-21 17:14|   41|
|          CA|May-21 17:14|  119|
|          US|May-21 17:14| 1152|
+------------+------------+-----+



Note the two ends of the graph. The close actions are generated such that they are after the corresponding open actions, so there are more "opens" in the beginning and more "closes" in the end.

## Stream Processing 
Now that we have analyzed the data interactively, let's convert this to a streaming query that continuously updates as data comes. Since we just have a static set of files, we are going to emulate a stream from them by reading one file at a time, in the chronological order they were created. The query we have to write is pretty much the same as the interactive query above.

In [9]:
from pyspark.sql.functions import *

# Similar to definition of staticInputDF above, just using `readStream` instead of `read`
streamingInputDF = (
  spark
    .readStream                       
    .schema(jsonSchema)               # Set the schema of the JSON data
    .option("maxFilesPerTrigger", 1)  # Treat a sequence of files as a stream by picking one file at a time
    .json(inputPath)
)

# Same query as staticInputDF
streamingCountsDF = (                 
  streamingInputDF
    .groupBy(
      streamingInputDF.country_code, 
      window(streamingInputDF.last_reported, "1 minute"))
    .count()
)

# Is this DF actually a streaming DF?
streamingCountsDF.isStreaming

True

As you can see, `streamingCountsDF` is a streaming Dataframe (`streamingCountsDF.isStreaming` was `true`). You can start streaming computation, by defining the sink and starting it. 
In our case, we want to interactively query the counts (same queries as above), so we will set the complete set of 1 hour counts to be in a in-memory table (note that this for testing purpose only in Spark 2.0).

In [10]:
spark.conf.set("spark.sql.shuffle.partitions", "2")  # keep the size of shuffles small

query = (
  streamingCountsDF
    .writeStream
    .format("memory")        # memory = store in-memory table (for testing only in Spark 2.0)
    .queryName("counts")     # counts = name of the in-memory table
    .outputMode("complete")  # complete = all the counts should be in the table
    .start()
)

`query` is a handle to the streaming query that is running in the background. This query is continuously picking up files and updating the windowed counts. 

Note the status of query in the above cell. Both the `Status: ACTIVE` and the progress bar shows that the query is active. 
Furthermore, if you expand the `>Details` above, you will find the number of files they have already processed. 

Let's wait a bit for a few files to be processed and then interactively query the in-memory `counts` table.

In [11]:
from time import sleep
sleep(2)  # wait a bit for computation to start

In [12]:
spark.sql("select country_code, date_format(window.end, 'MMM-dd HH:mm') as time, count from counts order by time, country_code").show()

+------------+------------+-----+
|country_code|        time|count|
+------------+------------+-----+
|          US|May-21 17:14|   40|
+------------+------------+-----+



We see the timeline of windowed counts (similar to the static one ealrier) building up. If we keep running this interactive query repeatedly, we will see the latest updated counts which the streaming query is updating in the background.

In [13]:
sleep(2)  # wait a bit more for more data to be computed

In [14]:
sleep(2)  # wait a bit more for more data to be computed
spark.sql("select country_code, date_format(window.end, 'MMM-dd HH:mm') as time, count from counts order by time, country_code").show()

+------------+------------+-----+
|country_code|        time|count|
+------------+------------+-----+
|          US|May-21 17:14|  139|
+------------+------------+-----+



Also, let's see the total number of "opens" and "closes".

In [ ]:
spark.sql("select country_code, sum(count) as total_count from counts group by country_code order by country_code").collect()

[Row(country_code='US', total_count=149)]

In [ ]:
# RUN a simple webservice on default port 5000 to return the in-memory table as is
from flask import Flask
import json
app = Flask(__name__)

# route to access data
@app.route("/")
def count_per_country():
    counts = spark.sql("select country_code, sum(count) as total_count from counts group by country_code").toJSON().collect()
    return json.dumps(counts)

if __name__ == "__main__":
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/May/2017 17:47:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2017 17:47:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2017 17:47:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2017 17:47:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2017 17:47:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2017 17:47:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2017 17:47:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2017 17:47:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2017 17:47:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2017 17:47:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2017 17:47:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2017 17:47:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2017 17:47:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2017 17:47:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2017 17:47:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2017 17:47:06] "GE

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 53613)
----------------------------------------


Traceback (most recent call last):
  File "/Users/mca/anaconda/envs/python3/lib/python3.5/socketserver.py", line 313, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/mca/anaconda/envs/python3/lib/python3.5/socketserver.py", line 341, in process_request
    self.finish_request(request, client_address)
  File "/Users/mca/anaconda/envs/python3/lib/python3.5/socketserver.py", line 354, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/mca/anaconda/envs/python3/lib/python3.5/socketserver.py", line 681, in __init__
    self.handle()
  File "/Users/mca/anaconda/envs/python3/lib/python3.5/site-packages/werkzeug/serving.py", line 232, in handle
    rv = BaseHTTPRequestHandler.handle(self)
  File "/Users/mca/anaconda/envs/python3/lib/python3.5/http/server.py", line 422, in handle
    self.handle_one_request()
  File "/Users/mca/anaconda/envs/python3/lib/python3.5/site-packages/werkzeug/serving.py", line 263, i